In [1]:
import numpy as np

import sensors
import em_stoch

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 20
Num_emitters1 = 1
sample_size1 = 80
failing_sensors1 = np.array([2, 4], dtype=np.int16)
gap_ratio1 = np.array([0.5, 0.5], dtype=np.float32)
theta1_rad = np.array([0.7]) # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q_1 = 2.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sensors.gss(Num_emitters1, sample_size1, P_1)
N1 = sensors.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + N1.T).T

Num_sensors2 = 20
Num_emitters2 = 2
sample_size2 = 80
failing_sensors2 = np.array([2, 4], dtype=np.int16)
gap_ratio2 = np.array([0.5, 0.5], dtype=np.float32)
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P_2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q_2 = 1.1 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sensors.gss(Num_emitters2, sample_size2, P_2)
N2 = sensors.gss(Num_sensors2, sample_size2, Q_2)
X2 = (A2 @ S2.T + N2.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, failing_sensors1, gap_ratio1)
X2_with_mv = sensors.MCAR(X2, failing_sensors2, gap_ratio2)

In [5]:
X1_with_mv

array([[ 1.47375889-1.19127215j,  0.03175204-1.88842948j,
                nan+0.j        , ..., -0.76397286-2.94873339j,
         0.72662444+0.27358696j,  1.62576741-0.61924096j],
       [-0.75093165-1.09293498j, -0.99980865-0.34039036j,
         2.94445197-1.56076135j, ...,  0.33293359+0.82505105j,
         0.07846728+0.13232699j, -1.07843768+0.39630101j],
       [ 0.76613778+1.51307616j,  0.98300291+0.8604167j ,
        -2.77695094-1.28910182j, ..., -0.60766186+0.05449384j,
        -1.0589133 -1.09211055j, -0.1291021 +1.6502334j ],
       ...,
       [ 1.16245983+2.06921586j,  2.97479457-1.66960229j,
        -2.74510029-0.87284081j, ...,  1.19492776-0.80840476j,
        -2.23063739-0.51135893j,  0.20866592+3.56230975j],
       [ 1.57700122+0.90672591j, -2.1236758 -2.06001117j,
        -1.06651492+2.87218098j, ..., -1.12889979-0.66445606j,
        -0.37962652-0.26983634j,  1.95978744-1.35519038j],
       [-0.56114001+1.07715026j, -0.59027711+0.12132465j,
                nan+0.j       

In [6]:
print(np.trace(sensors.initial_Cov(X1_with_mv)))
print(np.trace(sensors.complex_cov(X1)))

(73.88343486532314+0j)
(63.702954966635296+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em_stoch.multi_start_EM(X1_with_mv, 
                                                      Num_emitters1, 
                                                      Q=Q_1, 
                                                      num_of_starts=10, 
                                                      max_iter=30, 
                                                      rtol=1e-6)

0-th start
theta=[0.15335215],P=[[0.00022361+0.j]]
new_angles=[0.15335215]
new_P:
[[0.00038317+9.73127895e-18j]]
likelihood is -3420.8363426126166 on iteration 0
new_angles=[0.15335215]
new_P:
[[0.00038428+9.73136112e-18j]]
1-th start
theta=[0.13636065],P=[[0.0023446-2.48253415e-17j]]
new_angles=[0.13251507]
new_P:
[[0.02511729+3.98176759e-16j]]
likelihood is -3420.679471074612 on iteration 0
new_angles=[0.13172829]
new_P:
[[0.02795859+3.84142311e-16j]]
likelihood is -3420.8139649847435 on iteration 1
new_angles=[0.1310679]
new_P:
[[0.03147344+3.67926483e-16j]]
likelihood is -3421.023941079449 on iteration 2
new_angles=[0.13036965]
new_P:
[[0.03583874+3.49505448e-16j]]
likelihood is -3421.335351411021 on iteration 3
new_angles=[0.12969681]
new_P:
[[0.04127265+3.27950725e-16j]]
likelihood is -3421.794214749541 on iteration 4
new_angles=[0.12900761]
new_P:
[[0.04802238+3.03721171e-16j]]
likelihood is -3422.4547123590364 on iteration 5
new_angles=[0.12835657]
new_P:
[[0.05633836+2.7656705

In [8]:
print(f"EM_theta1_rad={EM_theta1_rad},\nEM_P1={EM_P1}")

EM_theta1_rad=[0.6998002],
EM_P1=[[2.02018025-4.66872041e-16j]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.01144755],
Погрешность в смысле разности синусов углов:       [0.00015283].


In [10]:
print(np.trace(sensors.initial_Cov(X2_with_mv)))
print(np.trace(sensors.complex_cov(X2)))

(49.65503774285381+0j)
(61.03171543408073+0j)


In [11]:
EM_theta2_rad, EM_P2, lhd_2 = em_stoch.multi_start_EM(X2_with_mv, 
                                                      Num_emitters2, 
                                                      Q=Q_2, 
                                                      num_of_starts=20, 
                                                      max_iter=30,
                                                      rtol=1e-3)

0-th start
theta=[-1.41744418  0.15335215],P=[[0.00276336-1.33017438e-17j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.00299515+1.51605636e-17j]]
new_angles=[-0.51963732  0.70927916]
new_P:
[[0.01710214-2.17596099e-17j 0.00048937-1.14722684e-02j]
 [0.00048937+1.14722684e-02j 0.02521894+6.18007553e-18j]]
likelihood is -3710.6181423562143 on iteration 0
new_angles=[-0.51472202  0.70718978]
new_P:
[[0.11372679-2.00252987e-17j 0.00448198-9.67769079e-02j]
 [0.00448198+9.67769079e-02j 0.1710447 +6.49565600e-18j]]
likelihood is -2819.6091242761104 on iteration 1
new_angles=[-0.51058971  0.70518426]
new_P:
[[ 7.32719687e-01-2.95958750e-17j -8.06434976e-04-4.56877991e-01j]
 [-8.06434976e-04+4.56877991e-01j  8.74243691e-01-2.20888026e-17j]]
likelihood is -2190.9874246303502 on iteration 2
new_angles=[-0.50805734  0.70398936]
new_P:
[[ 1.74277591+1.22828035e-16j -0.07355371-3.13262401e-01j]
 [-0.07355371+3.13262401e-01j  1.47701494+5.11996220e-16j]]
likelihood is -2143.8521441778134

In [12]:
print(f"EM_theta2_rad={EM_theta2_rad},\nEM_P2={EM_P2}")

EM_theta2_rad=[-0.50006197  0.70074579],
EM_P2=[[ 2.14529779e+00+4.85615857e-16j -1.84668126e-03-2.96551476e-01j]
 [-1.84668126e-03+2.96551476e-01j  1.71423350e+00-3.22791769e-16j]]


In [13]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.00355059 0.04273067],
Погрешность в смысле разности синусов углов:       [5.43824670e-05 5.70233168e-04].


In [14]:
em_stoch.incomplete_lkhd(X1_with_mv, theta1_rad, P_1, Q_1)

np.float64(-2824.8936593288777)

In [15]:
em_stoch.incomplete_lkhd(X1_with_mv, EM_theta1_rad, EM_P1, Q_1)

np.float64(-2835.831028878923)

In [16]:
em_stoch.incomplete_lkhd(X2_with_mv, theta2_rad, P_2, Q_2)

np.float64(-2106.4163518053156)

In [17]:
em_stoch.incomplete_lkhd(X2_with_mv, EM_theta2_rad, EM_P2, Q_2)

np.float64(-2131.796905303359)